In [1]:
# Build a formula evaluator

In [2]:
import pickle

def load_pickled_object(file_path):
    with open(file_path, 'rb') as file:
        obj = pickle.load(file)
    return obj

In [3]:
generic_formula_dictionary = load_pickled_object('generic_formula_dictionary.pkl')
series_dict = load_pickled_object('series_dict.pkl')

In [4]:
series_dict["Sheet1"]

[Series(series_id=SeriesId(sheet_name='Sheet1', series_header='col_1', series_header_cell_row=2, series_header_cell_column=2), worksheet=Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None), series_header='col_1', formulas=[None, None], values=[1, 3], header_location=<HeaderLocation.TOP: 'top'>, series_starting_cell=Cell(column=2, row=3, coordinate='B3', sheet_name=None, value=None, value_type=None, formula=None), series_length=2, series_data_type=<SeriesDataType.INT: 'int'>),
 Series(series_id=SeriesId(sheet_name='Sheet1', series_header='col_2', series_header_cell_row=2, series_header_cell_column=3), worksheet=Worksheet(sheet_name='Sheet1', workbook_file_path=None, worksheet=None), series_header='col_2', formulas=[None, None], values=[2, 4], header_location=<HeaderLocation.TOP: 'top'>, series_starting_cell=Cell(column=3, row=3, coordinate='C3', sheet_name=None, value=None, value_type=None, formula=None), series_length=2, series_data_type=<SeriesDataType.INT: 'int'>)

In [5]:
import ast

class FormulaEvaluator:

    def __init__(self, series_dict):
        self.local_env = {"SUM": self.SUM}

    def SUM(self):
        pass

    def evaluate_sum(self, sum_formula: str):
        tree = ast.parse(sum_formula, mode="eval")
        compiled = compile(tree, filename="<ast>", mode="eval")
        result = eval(compiled, {"__builtins__": {}}, self.local_env)
        return result


# NEW PLAN

1. Update the get formula list function so that there are arrays in the new formula, they look like this:

'(SUM(ARRAY(ARRAYROW(1, 2, 3)))) + (SUM(ARRAY(ARRAYROW(4, 5, 6), ARRAYROW(7, 8, 9))))'

2. Create an evaluator class that can evaluate this Excel formula directly in Python using the ast library.

OR

Investigate whether I can use the xcalculator library to create an Excel formula evaluator.


